In [ ]:
import ray
#import ray.services
from ray import workflow
from typing import List

if ray.is_initialized():
    pass
else:
    ray.init(dashboard_host="0.0.0.0")

    @workflow.step
    def read_data(num: int):
        return [i for i in range(num)]

    @workflow.step
    def preprocessing(data: List[float]) -> List[float]:
        return [d**2 for d in data]

    @workflow.step
    def aggregate(data: List[float]) -> float:
        return sum(data)

    # Initialize workflow storage.
    #workflow.init()
    workflow.init(storage="/tmp/data")


: 

In [2]:
import socket
ip = socket.gethostbyname(socket.gethostname())
dashboard = f"http://{ip}:8265"
print(dashboard)

http://192.168.72.106:8265


In [3]:

# Setup the workflow.
data = read_data.step(10)
preprocessed_data = preprocessing.step(data)
output = aggregate.step(preprocessed_data)


In [4]:
ref = output.run_async()

2022-04-22 21:57:20,076	INFO execution.py:49 -- Workflow job created. [id="b25b0e16-cde8-4330-a817-4e1afc0c4d98.1650635840.072602987", storage_url="file:///tmp/ray/workflow_data"]. Type: FUNCTION.


RuntimeError: This event loop is already running

In [7]:
@workflow.step
def add(left: int, right: int) -> int:
    return left + right

@workflow.step
def get_val() -> int:
    return 10

ret = add.step(get_val.step(), 20)
#assert ret.run("add_example") == 30


In [8]:
ret.run("add example")


2022-04-22 22:04:33,858	INFO execution.py:49 -- Workflow job created. [id="add example", storage_url="file:///tmp/ray/workflow_data"]. Type: FUNCTION.
/Users/ziyu4huang/miniforge3/envs/ray_tf2d7/lib/python3.9/site-packages/pygments/regexopt.py:77: RuntimeWarning: coroutine 'WorkflowStorage._put' was never awaited
  '|'.join(regex_opt_inner(list(group[1]), '')


RuntimeError: This event loop is already running

In [9]:
@workflow.step
def double(v):
    return 2 * v

inner_step = double.options(name="inner").step(1)
outer_step = double.options(name="outer").step(inner_step)
result = outer_step.run_async("double")
inner = workflow.get_output("double", name="inner")
outer = workflow.get_output("double", name="outer")

assert ray.get(inner) == 2
assert ray.get(outer) == 4
assert ray.get(result) == 4

2022-04-22 22:05:07,566	INFO execution.py:49 -- Workflow job created. [id="double", storage_url="file:///tmp/ray/workflow_data"]. Type: FUNCTION.


RuntimeError: This event loop is already running

sys:1: RuntimeWarning: coroutine 'WorkflowStorage._put' was never awaited


In [2]:

assert output.run(workflow_id="run_1") == 101
assert workflow.get_status("run_1") == workflow.WorkflowStatus.SUCCESSFUL
assert workflow.get_output("run_1") == 101

NameError: name 'output' is not defined

In [4]:
@ray.remote
def hello():
    return "hello"

@workflow.step
def words() -> List[ray.ObjectRef]:
    return [hello.remote(), ray.put("world")]

@workflow.step
def concat(words: List[ray.ObjectRef]) -> str:
    return " ".join([ray.get(w) for w in words])

#workflow.init()
#assert concat.step(words.step()).run() == "hello world"

concat.step(words.step()).run()

2022-04-22 22:10:13,518	INFO execution.py:49 -- Workflow job created. [id="6da000b5-3915-4068-ad99-b77dce572f66.1650636613.517577887", storage_url="file:///tmp/data"]. Type: FUNCTION.


RuntimeError: This event loop is already running

In [5]:
concat.step(words.step()).run()

2022-04-22 22:10:25,477	INFO execution.py:49 -- Workflow job created. [id="4edeaf1a-0ec2-4259-b310-39a6916c0b2a.1650636625.476421833", storage_url="file:///tmp/data"]. Type: FUNCTION.


RuntimeError: This event loop is already running

In [6]:
nest_asyncio.apply()

NameError: name 'nest_asyncio' is not defined

sys:1: RuntimeWarning: coroutine 'WorkflowStorage._put' was never awaited


In [9]:
from ray import workflow
import ray

@workflow.virtual_actor
class Counter:
    def __init__(self, init_val):
        self._val = init_val

    def incr(self, val=1):
        self._val += val
        print(self._val)

    @workflow.virtual_actor.readonly
    def value(self):
        return self._val

#workflow.init()


In [10]:

# Initialize a Counter actor with id="my_counter".
counter = Counter.get_or_create("my_counter", 0)

# Similar to workflow steps, actor methods support:
# - `run()`, which will return the value
# - `run_async()`, which will return a ObjectRef
counter.incr.run(10)
assert counter.value.run() == 10

# Non-blocking execution.
counter.incr.run_async(10)
counter.incr.run(10)
assert 30 == ray.get(counter.value.run_async())

RuntimeError: This event loop is already running

In [17]:
from ray import workflow
import time

# Create an event which finishes after 60 seconds.
event1_step = workflow.wait_for_event(workflow.event_listener.TimerListener, time.time() + 60)

# Create another event which finishes after 30 seconds.
event2_step = workflow.wait_for_event(workflow.event_listener.TimerListener, time.time() + 30)

@workflow.step
def gather(*args):
    return args;

# Gather will run after 60 seconds, when both event1 and event2 are done.
gather.step(event1_step, event2_step).run()

2022-04-22 22:29:25,550	INFO execution.py:49 -- Workflow job created. [id="89a10eff-5a6b-454a-b5d2-e9f650d90982.1650637765.534621000", storage_url="file:///tmp/data"]. Type: FUNCTION.


RuntimeError: This event loop is already running